In [1]:
%load_ext autoreload
%autoreload 2
import os; import sys;sys.path.append("..")
import pandas as pd
import tqdm
import socceraction.spadl as spadl

AttributeError: module 'socceraction' has no attribute 'spadl'

In [ ]:
import urllib

url = "https://github.com/statsbomb/open-data/archive/master.zip"
urllib.request.urlretrieve(url, '../data/statsbomb-open-data.zip')

In [ ]:
import zipfile

with zipfile.ZipFile('../data/statsbomb-open-data.zip', 'r') as zipObj:
    zipObj.extractall('../data')

In [ ]:
%%time
spadl.spadl_length
#statsbomb.jsonfiles_to_h5("../data/open-data-master/data","../data/statsbomb.h5")

In [ ]:
matches = pd.read_hdf("../data/statsbomb.h5","matches")
match_id = matches.match_id[0]
events = pd.read_hdf("../data/statsbomb.h5",f'events/match_{match_id}')
matches
#events